In [ ]:
import pickle as pkl
from collections import namedtuple
from datetime import datetime, timedelta
from functools import lru_cache 
import json
from urllib.request import urlopen

from bs4 import BeautifulSoup

In [ ]:
soup = BeautifulSoup(open("messages.htm").read(), 'html.parser')

In [ ]:
me = "" # YOUR NAME HERE
API_KEY = "" # YOUR sex-API KEY HERE 

Message = namedtuple("Message", ['person', 'sent_by_me', 'timestamp', 'sex']) 
                         # types: str,      bool,         datetime,     str

In [ ]:
@lru_cache(maxsize=None)
def get_sex(name):
    
    if len(name.split(" ")) != 1:
        name = name.split(" ")[0]
        name = str(name.encode('ascii', 'ignore'))
        
    myKey = API_KEY
    url = "https://gender-api.com/get?key=" + myKey + "&name=" + name

    response = urlopen(url)
    decoded = response.read().decode('utf-8')
    data = json.loads(decoded)

    return data["sex"]

In [ ]:
divs = soup.findAll('div', class_="thread")

In [ ]:
threads = [] 
messages = []

for thread in divs:
    #print(thread.contents)
    
    people = list(map(str.strip, thread.contents[0].split(',')))
    
    if 2 != len(people): # skip group chats for now
        continue
    
    person1, person2 = people
    person = person1 if person2 == me else person2 # who im talking to 
            
    
    for item in thread.contents[1:]:

        if item.name == "div" and item["class"][0] == "message":
            
            datestring = item.contents[0].contents[1].contents[0]
            timestamp = datetime.strptime(datestring, '%A, %B %d, %Y at %I:%M%p %Z')
            
            person_sending = item.contents[0].contents[0].contents[0]
            sent_by_me = True if person_sending == me else False
            
            sex = get_sex(person)
            
            messages.append(Message(person, sent_by_me, timestamp, sex))
            

In [ ]:
pkl.dump(messages, open("messages.pkl", "wb"))